In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', None)

In [2]:
## importing of data and cleaning up of dataframe

chess2021 = pd.read_csv('Chess2021.csv')
chess2021['Elo Difference'] = abs(chess2021.WhiteElo - chess2021.BlackElo)
chess2021outcomes = chess2021[['Event', 'Date', 'White', 'Black', 'Result', 'WhiteElo', 'BlackElo', 'Elo Difference', 'ECO', 'NMoves']]

eventList = []
for i in chess2021outcomes.Event:
    if i == 'FICS rated blitz game':
        i = 'Rated Blitz'
        eventList.append(i)
    elif i == 'FICS rated lightning game':
        i = 'Rated Bullet'
        eventList.append(i)
    elif i == 'FICS rated standard game':
        i = 'Rated Standard'
        eventList.append(i)
    elif i == 'FICS unrated standard game':
        i = 'Unrated Standard'
        eventList.append(i)
    elif i == 'FICS unrated blitz game':
        i = 'Unrated Blitz'
        eventList.append(i)
    else:
        i = 'Unrated Bullet'
        eventList.append(i)
chess2021outcomes.Event = eventList

resultList = []
for i in chess2021outcomes.Result:
    if i == '1-0':
        i = 1
        resultList.append(i)
    elif i == '0-1':
        i = -1
        resultList.append(i)
    else:
        i = 0
        resultList.append(i)
chess2021outcomes.Result = resultList

chess2021outcomes

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74
...,...,...,...,...,...,...,...,...,...,...
118255,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64
118256,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129
118257,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75
118258,Unrated Standard,2021.01.01,Notarious,ArasanX,0,2917,2669,248,C89,115


In [3]:
## explanation of elo rating change calculation

# k rating calculation
# 20 if standard rating below 2400
# 10 is standard rating at or above 2400
# 20 for all rapid/blitz ratings

# WhiteElo - BlackElo = A
# A/400 = B
# (10^B) + 1 = C
# 1/C = D
# --if higher elo wins
# kRating*(D) = Final Rating Change (rounded to whole number)
# --if lower elo wins
# kRating*(1-D) = Final Rating Change (rounded to whole number)

chess2021outcomes

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74
...,...,...,...,...,...,...,...,...,...,...
118255,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64
118256,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129
118257,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75
118258,Unrated Standard,2021.01.01,Notarious,ArasanX,0,2917,2669,248,C89,115


In [4]:
finalRatingChangeList = []
finalWhiteEloList = []
finalBlackEloList = []

for i, j, k, l in zip(chess2021outcomes.WhiteElo, chess2021outcomes.BlackElo, chess2021outcomes.Result, chess2021outcomes.Event):
    if l == 'Rated Blitz' or l == 'Rated Bullet':
        kRating = 20
    elif l == 'Rated Standard':
        if i > 2400 or j > 2400:
            kRating = 10
        else:
            kRating = 20
    else:
        kRating = 0

    eloDifference = abs(i-j)
    x = eloDifference/400
    y = (10**x) + 1
    z = 1/y

    finalRatingChange = 0
    if i > j:
        if k == 1:
            finalRatingChange = round(kRating*z, 1)
            i = round(i + finalRatingChange)
            j = round(j - finalRatingChange)
        elif k == -1:
            finalRatingChange = round(kRating*(1-z), 1)
            i = round(i - finalRatingChange)
            j = round(j + finalRatingChange)
        else:
            finalRatingChange = round(kRating*(0.5-z), 1)
            i = round(i - finalRatingChange)
            j = round(j + finalRatingChange)
    elif j > i:
        if k == 1:
            finalRatingChange = round(kRating*(1-z), 1)
            i = round(i + finalRatingChange)
            j = round(j - finalRatingChange)
        elif k == -1:
            finalRatingChange = round(kRating*z, 1)
            i = round(i - finalRatingChange)
            j = round(j + finalRatingChange)
        else:
            finalRatingChange = round(kRating*(0.5-z), 1)
            i = round(i + finalRatingChange)
            j = round(j - finalRatingChange)
    else:
        if kRating == 0:
            finalRatingChange = 0
            i = i
            j = j
        elif kRating == 20:
            if k == 1:
                finalRatingChange = 10
                i = round(i + finalRatingChange)
                j = round(j - finalRatingChange)
            elif k == -1:
                finalRatingChange = 10
                i = round(i - finalRatingChange)
                j = round(j + finalRatingChange)
            else:
                finalRatingChange = 0
                i = i
                j = j
        else:
            if k == 1:
                finalRatingChange = 5
                i = round(i + finalRatingChange)
                j = round(j - finalRatingChange)
            elif k == -1:
                finalRatingChange = 5
                i = round(i - finalRatingChange)
                j = round(j + finalRatingChange)
            else:
                finalRatingChange = 0
                i = i
                j = j

    finalRatingChangeList.append(finalRatingChange)
    finalWhiteEloList.append(i)
    finalBlackEloList.append(j)

In [5]:
whiteElo = 2405
blackElo = 2422
outcome = 1
event = 'Rated Blitz'

if event == 'Rated Blitz' or event == 'Rated Bullet':
    kRating = 20
elif event == 'Rated Standard':
    if whiteElo > 2400 or blackElo > 2400:
        kRating = 10
    else:
        kRating = 20
else:
    kRating = 0

eloDifference = abs(whiteElo-blackElo)
x = eloDifference/400
y = (10**x) + 1
z = 1/y

finalRatingChange = 0
if whiteElo > blackElo:
    if outcome == 1:
        finalRatingChange = round(kRating*z, 1)
        whiteElo = round(whiteElo + finalRatingChange)
        blackElo = round(blackElo - finalRatingChange)
    elif outcome == -1:
        finalRatingChange = round(kRating*(1-z), 1)
        whiteElo = round(whiteElo - finalRatingChange)
        blackElo = round(blackElo + finalRatingChange)
    else:
        finalRatingChange = round(kRating*(0.5-z), 1)
        whiteElo = round(whiteElo - finalRatingChange)
        blackElo = round(blackElo + finalRatingChange)
elif blackElo > whiteElo:
    if outcome == 1:
        finalRatingChange = round(kRating*(1-z), 1)
        whiteElo = round(whiteElo + finalRatingChange)
        blackElo = round(blackElo - finalRatingChange)
    elif outcome == -1:
        finalRatingChange = round(kRating*z, 1)
        whiteElo = round(whiteElo - finalRatingChange)
        blackElo = round(blackElo + finalRatingChange)
    else:
        finalRatingChange = round(kRating*(0.5-z), 1)
        whiteElo = round(whiteElo + finalRatingChange)
        blackElo = round(blackElo - finalRatingChange)
else:
    if kRating == 0:
        finalRatingChange = 0
        whiteElo = whiteElo
        blackElo = blackElo
    elif kRating == 20:
        if outcome == 1:
            finalRatingChange = 10
            whiteElo = round(whiteElo + finalRatingChange)
            blackElo = round(blackElo - finalRatingChange)
        elif outcome == -1:
            finalRatingChange = 10
            whiteElo = round(whiteElo - finalRatingChange)
            blackElo = round(blackElo + finalRatingChange)
        else:
            finalRatingChange = 0
            whiteElo = whiteElo
            blackElo = blackElo
    else:
        if outcome == 1:
            finalRatingChange = 5
            whiteElo = round(whiteElo + finalRatingChange)
            blackElo = round(blackElo - finalRatingChange)
        elif outcome == -1:
            finalRatingChange = 5
            whiteElo = round(whiteElo - finalRatingChange)
            blackElo = round(blackElo + finalRatingChange)
        else:
            finalRatingChange = 0
            whiteElo = whiteElo
            blackElo = blackElo

In [6]:
chess2021outcomes['FinalWhiteElo'] = finalWhiteEloList
chess2021outcomes['FinalBlackElo'] = finalBlackEloList
chess2021outcomes['RatingChange'] = finalRatingChangeList

chess2021outcomes

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves,FinalWhiteElo,FinalBlackElo,RatingChange
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14,2089,2013,8.3
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48,1932,2078,14.7
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26,2094,1916,5.5
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27,2750,2752,10.5
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74,2742,2760,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118255,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64,2738,2878,6.5
118256,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129,2871,2746,3.7
118257,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75,2740,2877,6.6
118258,Unrated Standard,2021.01.01,Notarious,ArasanX,0,2917,2669,248,C89,115,2917,2669,0.0


In [7]:
chess2021outcomesRatedBlitz = chess2021outcomes.loc[chess2021outcomes.Event == 'Rated Blitz'].reset_index(drop=True)
chess2021outcomesRatedBlitz

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves,FinalWhiteElo,FinalBlackElo,RatingChange
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14,2089,2013,8.3
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48,1932,2078,14.7
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26,2094,1916,5.5
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27,2750,2752,10.5
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74,2742,2760,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51873,Rated Blitz,2021.01.01,MIHAILOP,PetkoPetkov,1,2253,1987,266,B23,36,2257,1983,3.6
51874,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64,2738,2878,6.5
51875,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129,2871,2746,3.7
51876,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75,2740,2877,6.6


In [8]:
ecoList = list(chess2021outcomes.ECO.unique())
ecoList.sort()
ecoList

['A00',
 'A01',
 'A02',
 'A03',
 'A04',
 'A05',
 'A06',
 'A07',
 'A08',
 'A09',
 'A10',
 'A11',
 'A12',
 'A13',
 'A14',
 'A15',
 'A16',
 'A17',
 'A18',
 'A19',
 'A20',
 'A21',
 'A22',
 'A23',
 'A24',
 'A25',
 'A26',
 'A27',
 'A28',
 'A29',
 'A30',
 'A31',
 'A32',
 'A33',
 'A34',
 'A35',
 'A36',
 'A37',
 'A38',
 'A39',
 'A40',
 'A41',
 'A42',
 'A43',
 'A44',
 'A45',
 'A46',
 'A47',
 'A48',
 'A49',
 'A50',
 'A51',
 'A52',
 'A53',
 'A54',
 'A55',
 'A56',
 'A57',
 'A58',
 'A59',
 'A60',
 'A61',
 'A62',
 'A63',
 'A64',
 'A65',
 'A66',
 'A67',
 'A68',
 'A69',
 'A70',
 'A72',
 'A73',
 'A74',
 'A75',
 'A76',
 'A77',
 'A78',
 'A79',
 'A80',
 'A81',
 'A82',
 'A83',
 'A84',
 'A85',
 'A86',
 'A87',
 'A88',
 'A89',
 'A90',
 'A91',
 'A92',
 'A93',
 'A94',
 'A95',
 'A96',
 'A97',
 'A98',
 'A99',
 'B00',
 'B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14',
 'B15',
 'B16',
 'B17',
 'B18',
 'B19',
 'B20',
 'B21',
 'B22',
 'B23',
 'B24',
 'B25',


In [9]:
## webscrape/pull ECO (chess openings)
url = 'https://www.365chess.com/eco/'

ecoName = []
for i in ecoList:
    response = requests.get(url + str(i))
    soup = bs(response.text, 'html.parser')
    ecoName.append(soup.findAll('h1'))
    time.sleep(5)
    print(i)

A00
A01
A02
A03
A04
A05
A06
A07
A08
A09
A10
A11
A12
A13
A14
A15
A16
A17
A18
A19
A20
A21
A22
A23
A24
A25
A26
A27
A28
A29
A30
A31
A32
A33
A34
A35
A36
A37
A38
A39
A40
A41
A42
A43
A44
A45
A46
A47
A48
A49
A50
A51
A52
A53
A54
A55
A56
A57
A58
A59
A60
A61
A62
A63
A64
A65
A66
A67
A68
A69
A70
A72
A73
A74
A75
A76
A77
A78
A79
A80
A81
A82
A83
A84
A85
A86
A87
A88
A89
A90
A91
A92
A93
A94
A95
A96
A97
A98
A99
B00
B01
B02
B03
B04
B05
B06
B07
B08
B09
B10
B11
B12
B13
B14
B15
B16
B17
B18
B19
B20
B21
B22
B23
B24
B25
B26
B27
B28
B29
B30
B31
B32
B33
B34
B35
B36
B37
B38
B39
B40
B41
B42
B43
B44
B45
B46
B47
B48
B49
B50
B51
B52
B53
B54
B55
B56
B57
B58
B59
B60
B61
B62
B63
B64
B65
B66
B67
B68
B69
B70
B71
B72
B73
B74
B75
B76
B77
B78
B79
B80
B81
B82
B83
B84
B85
B86
B87
B88
B89
B90
B91
B92
B93
B94
B95
B96
B97
B98
B99
C00
C01
C02
C03
C04
C05
C06
C07
C08
C09
C10
C11
C12
C13
C14
C15
C16
C17
C18
C19
C20
C21
C22
C23
C24
C25
C26
C27
C28
C29
C30
C31
C32
C33
C34
C35
C36
C37
C38
C39
C40
C41
C42
C43
C44
C45
C46
C47
C48
C49
C50


In [10]:
ecoName

[[<h1>A00: Polish (Sokolsky) opening</h1>],
 [<h1>A01: Nimzovich-Larsen attack</h1>],
 [<h1>A02: Bird's opening</h1>],
 [<h1>A03: Bird's opening</h1>],
 [<h1>A04: Reti opening</h1>],
 [<h1>A05: Reti opening</h1>],
 [<h1>A06: Reti opening</h1>],
 [<h1>A07: Reti, King's Indian attack (Barcza system)</h1>],
 [<h1>A08: Reti, King's Indian attack</h1>],
 [<h1>A09: Reti opening</h1>],
 [<h1>A10: English opening</h1>],
 [<h1>A11: English, Caro-Kann defensive system</h1>],
 [<h1>A12: English, Caro-Kann defensive system</h1>],
 [<h1>A13: English opening</h1>],
 [<h1>A14: English, Neo-Catalan declined</h1>],
 [<h1>A15: English, 1...Nf6 (Anglo-Indian defense)</h1>],
 [<h1>A16: English opening</h1>],
 [<h1>A17: English opening</h1>],
 [<h1>A18: English, Mikenas-Carls variation</h1>],
 [<h1>A19: English, Mikenas-Carls, Sicilian variation</h1>],
 [<h1>A20: English opening</h1>],
 [<h1>A21: English opening</h1>],
 [<h1>A22: English opening</h1>],
 [<h1>A23: English, Bremen system, Keres variation</h1